In [1]:
from sklearn.mixture import GaussianMixture
import numpy as np
from utils import Givens2Matrix, QRGivens, eigh_with_fixed_direction_range, find_closest_spd
from utils import load_cloud_dataset, load_breast_cancer, load_seg_data, load_digits_dataset, load_satelite_dataset, load_synthetic_dataset
from python_example import Givens2Matrix_double as Givens2Matrix
from python_example import QRGivens_double as QRGivens
from copy import deepcopy

In [5]:
import warnings
# from sklearn.exceptions import ConvergenceWarning, ComplexWarning
# warnings.filterwarnings(action='ignore', category=ComplexWarning)
# warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

In [19]:

class EMParticle:
    def __init__(self, weights, means, precision_matrices, eigvals_coef, means_coef) -> None:
        self.weights = deepcopy(weights)
        self.means = deepcopy(means)
        self.precision_matrices = deepcopy(precision_matrices)
        self.eigvals_coef = eigvals_coef
        self.n_comp = self.weights.shape[0]
        self.data_dim = self.means.shape[1]
        self.means_coef = means_coef

    def inject_noise(self):
        
        eigvals = [np.mean(np.linalg.eigvals(self.precision_matrices[i])) for i in range(self.n_comp)]
        
        eig_val_max  = [eigvals[i] * self.eigvals_coef for i in range(len(eigvals))]

        for i in range(self.n_comp):
            
            givens_angles = np.random.uniform(-np.pi, np.pi, size=(int(self.data_dim * (self.data_dim - 1) / 2)))
            delta_eigvals = np.random.uniform(0, np.mean(eig_val_max), size=self.data_dim)
            # print('eig', np.mean(eig_val_max))
            
            v = Givens2Matrix(np.expand_dims(givens_angles, axis=1))
            addition = v @ np.diag(delta_eigvals) @ v.T

            self.precision_matrices += addition
        # print(np.mean(self.means))
        means_delta = np.random.normal(0, self.means_coef * np.mean(self.means), size=(self.n_comp, self.data_dim))
        # print(self.means.shape, means_delta.shape)
        self.means += means_delta

    def run_em_iters(self, iters, data):
        gmm = GaussianMixture(n_components=self.weights.shape[0], covariance_type='full',
         weights_init=self.weights, means_init=self.means, precisions_init=self.precision_matrices, max_iter=iters)
        gmm.fit(data)
        self.weights  = gmm.weights_
        self.means = gmm.means_
        self.precision_matrices = gmm.precisions_
        self.curr_score = gmm.score(data)
        return gmm.score(data)

class NoiseEM:
    def __init__(self, n_comp, n_particles, T1, T2, eigval_coef, means_coef) -> None:
        self.particles = [None for i in range(n_particles)]
        self.n_comp = n_comp
        self.T1 = T1
        self.T2 = T2
        self.eigval_coef = eigval_coef
        self.means_coef = means_coef

    def run(self, data):
        best_score = -np.inf
        best_particle = None
        for i in range(len(self.particles)):
            particle_gmm = GaussianMixture(self.n_comp, covariance_type='full', max_iter=1, n_init=1, init_params='k-means++')
            particle_gmm.fit(data)
            self.particles[i] = EMParticle(particle_gmm.weights_, particle_gmm.means_, particle_gmm.precisions_, self.eigval_coef, self.means_coef)
            
        for i in range(self.T1):
            for particle in self.particles:
                ll = particle.run_em_iters(self.T2, data)
                print('LL: ', ll)
                if ll > best_score:
                    best_score = ll
                    best_particle = deepcopy(particle)
                # particle.inject_noise()
            
            for j in range(len(self.particles)):
                self.particles[j] = EMParticle(best_particle.weights, best_particle.means, best_particle.precision_matrices, self.eigval_coef, self.means_coef)
                self.particles[j].inject_noise()

        for particle in self.particles:
            ll = particle.run_em_iters(self.T2, data)
            if ll > best_score:
                best_score = ll
            
            print('Final LL: ', ll)
        print('Best LL: ', best_score)

In [49]:
from utils import load_dataset
data = load_seg_data()
data = load_dataset('seg')
T1 = 10
T2 = 10
n_comp = 10
n_particles = 30

eigvals_coef = 0.1
means_coef = 0.1

em = NoiseEM(n_comp, n_particles, T1, T2, eigvals_coef, means_coef)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    em.run(data)


LL:  68.76140762766123
LL:  69.96093803681335
LL:  65.70113094132631
LL:  65.99577943526985
LL:  68.98821761327493
LL:  69.90470355816441
LL:  68.87191888298712
LL:  70.71383344220318
LL:  69.93888553783968
LL:  69.83436427776095
LL:  67.60871866423422
LL:  69.32422934184835
LL:  67.62682344094124
LL:  67.50021340833462
LL:  69.94159302829652
LL:  66.57129947053618
LL:  63.705404580706826
LL:  66.17205537499484
LL:  66.79285786770336
LL:  68.37075408374385
LL:  69.53725555360846
LL:  69.9330391107168
LL:  69.65519640208119
LL:  66.5243678268894
LL:  69.4290430834794
LL:  67.52111883317822
LL:  65.90418773398719
LL:  68.29158446036068
LL:  67.27426019327044
LL:  68.80644876367936
LL:  71.16452135108099
LL:  70.87735130139015
LL:  70.89188193852779
LL:  71.22662477663255
LL:  71.0628910775597
LL:  71.16985448367994
LL:  71.21447098659723
LL:  71.06289136168778
LL:  70.95985266553399
LL:  71.21137711386952
LL:  71.17112289955782
LL:  71.08738583695069
LL:  71.18108889595295
LL:  70.986470

In [46]:

ref_gmm = GaussianMixture(n_comp, n_init=int(n_particles * T1 / 3), max_iter=T2 * 3, verbose=0, verbose_interval=1)
ref_gmm.fit(data)
ref_gmm.score(data)


71.43196036534148